In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [5]:
t = pd.read_csv('total_reviews.csv')

In [6]:
t.head()

,Unnamed: 0,Unnamed: 0.1,asin,product,date,verified,title,desc,reviewer_name,rating
0,0,0,B08267BBJT,Size: AAA,"August 11, 2020",Verified Purchase,"\nDidn't work, then worked, now don't work aga...",\n\n All I got in terms of use out of these b...,Jasmine Carroll,1.0
1,1,1,B08267BBJT,Size: AAA,"July 30, 2020",Verified Purchase,\nThese absolutely suck\n,\n\n I bought these for a wall mounted magnif...,Ashlee M.,1.0
2,2,2,B08268F6XN,Size: AA,"July 19, 2020",Verified Purchase,\nlonger lasting battery for remote controller...,\n\n i like the constant voltage and hopefull...,ARCHANGEL TROY,5.0
3,3,3,B08267BBJT,Size: AAA,"July 18, 2020",Verified Purchase,\nMinimal plastic in packaging.\n,"\n\n Just received these today, but I’m revie...",ira,5.0
4,4,4,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,\nNot long enough battery life for a night hike\n,\n\n Shuts off suddenly in headlamp\n\n,T,3.0


In [2]:
asin_url = 'https://www.amazon.com/dp/B08267BBJT'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [12]:
reviews_url = 'https://www.amazon.com/Rechargeable-Batteries-Charger-Charging-Indicator/product-reviews/B08267BBJT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'
rev = requests.get(reviews_url, headers=headers)
reviews_page_content = rev.content

In [13]:
reviews_page_content

b'<!doctype html><html lang="en-us" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-na.ssl-images-amazon.com" crossorigin>\n<link rel="preconnect" href="https://images-na.ssl-images-amazon.com" crossorigin>\n<link rel="dns-prefetch" href="https://m.media-amazon.com" crossorigin>\n<link rel="preconnect" href="https://m.media-amazon.com" crossorigin>\n<link rel="dns-prefetch" href="https://completion.amazon.com" crossorigin>\n<link rel="preconnect" href="https://completion.amazon.com" crossorigin>\n<script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Dat

In [6]:
review_soup = BeautifulSoup(reviews_page_content)

In [7]:
review_soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<script>var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<script>
(function(a){var b=a.performance;a.ue_mark=b&&"function"===typeof b.mark?function(a){b.mark(a)}:function(){}})(window);

    ue_mark('csm:ho:ob');
</script>
<script>
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;

if (window.ue_ihb === 1) {

    var ue_csm = window,
        ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c

Remember to check for <div class="a-section a-spacing-top-large a-text-center no-reviews-section"

In [8]:
reviews = []
asin = 'B08267BBJT'

for d in review_soup.findAll('div', attrs={'data-hook':'review'}):
#         print(d)
        date = d.find('span', attrs={'data-hook':'review-date'})
        date = date.text.split(' ')[-3:]
        date = ' '.join(date)
        title = d.find('a', attrs={'data-hook': 'review-title'})
        product = d.find('a', attrs={'data-hook': 'format-strip'})
        review_asin = product['href'].split('/')[3]
        product = product.text
        verified = d.find('span', attrs={'data-hook':'avp-badge'})
        if verified == None:
            verified = 'Not Verified'
        else:
            verified = verified.text
#         print(title['href'][-10:])
        description = d.find('span', attrs={'data-hook': 'review-body'})
        reviewer_name = d.find('span', attrs={'class': 'a-profile-name'})
        stars = d.find('span', attrs={'class': 'a-icon-alt'})
        reviews.append([review_asin, product, date, verified, title.text, description.text, reviewer_name.text, float(stars.text[0:3])])
#         n = name.find_all('img', alt=True)
#         #print(n[0]['alt'])
#         author = d.find('a', attrs={'class':'a-size-small a-link-child'})
#         rating = d.find('span', attrs={'class':'a-icon-alt'})
#         users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
#         price = d.find('span', attrs={'class':'p13n-sc-price'})

In [9]:
p = 'Reviewed in the United States on January 23, 2020'
p

'Reviewed in the United States on January 23, 2020'

In [10]:
reviews_df = pd.DataFrame(reviews, columns=['asin','product','date', 'verified', 'title', 'desc', 'reviewer_name', 'rating'])

In [11]:
reviews_df

,asin,product,date,verified,title,desc,reviewer_name,rating
0,B08267BBJT,Size: AAA,"August 11, 2020",Verified Purchase,"\nDidn't work, then worked, now don't work aga...",\n\n All I got in terms of use out of these b...,Jasmine Carroll,1.0
1,B08267BBJT,Size: AAA,"July 30, 2020",Verified Purchase,\nThese absolutely suck\n,\n\n I bought these for a wall mounted magnif...,Ashlee M.,1.0
2,B08268F6XN,Size: AA,"July 19, 2020",Verified Purchase,\nlonger lasting battery for remote controller...,\n\n i like the constant voltage and hopefull...,ARCHANGEL TROY,5.0
3,B08267BBJT,Size: AAA,"July 18, 2020",Verified Purchase,\nMinimal plastic in packaging.\n,"\n\n Just received these today, but I’m revie...",ira,5.0
4,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,\nNot long enough battery life for a night hike\n,\n\n Shuts off suddenly in headlamp\n\n,T,3.0
5,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,"\nSo far, I love them!\n",\n\n I bought these because we were burning t...,Joseph M Zenevitch,5.0
6,B08268F6XN,Size: AA,"July 16, 2020",Not Verified,\nNever want a disposable AA ever again\n,\n\n I didn't realize how barbaric it was to ...,M. L. Kijewski,5.0
7,B08267BBJT,Size: AAA,"July 1, 2020",Verified Purchase,\nEnergy Storage Is Less Than 1/4 Normal Batte...,"\n\n They do charge quickly, but at what pric...",Brent R.,1.0
8,B08267BBJT,Size: AAA,"June 28, 2020",Verified Purchase,\nPoor Quality - Unreliable\n,\n\n I had high hopes for these batteries bec...,Christian D.,1.0
9,B08267X3LH,Size: 9V,"June 12, 2020",Verified Purchase,\nGet these batteries!\n,\n\n I can’t believe that I can love a batter...,techie511,5.0


In [13]:
no_reviews_url = 'https://www.amazon.com/Rechargeable-Batteries-Charger-Charging-Indicator/product-reviews/B08267BBJT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=12'
no_rev = requests.get(no_reviews_url, headers=headers)

In [24]:
new_reviews_df = reviews_df.append(reviews_df)

In [25]:
new_reviews_df

,asin,product,date,verified,title,desc,reviewer_name,rating
0,B08267BBJT,Size: AAA,"August 11, 2020",Verified Purchase,"\nDidn't work, then worked, now don't work aga...",\n\n All I got in terms of use out of these b...,Jasmine Carroll,1.0
1,B08267BBJT,Size: AAA,"July 30, 2020",Verified Purchase,\nThese absolutely suck\n,\n\n I bought these for a wall mounted magnif...,Ashlee M.,1.0
2,B08268F6XN,Size: AA,"July 19, 2020",Verified Purchase,\nlonger lasting battery for remote controller...,\n\n i like the constant voltage and hopefull...,ARCHANGEL TROY,5.0
3,B08267BBJT,Size: AAA,"July 18, 2020",Verified Purchase,\nMinimal plastic in packaging.\n,"\n\n Just received these today, but I’m revie...",ira,5.0
4,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,\nNot long enough battery life for a night hike\n,\n\n Shuts off suddenly in headlamp\n\n,T,3.0
5,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,"\nSo far, I love them!\n",\n\n I bought these because we were burning t...,Joseph M Zenevitch,5.0
6,B08268F6XN,Size: AA,"July 16, 2020",Not Verified,\nNever want a disposable AA ever again\n,\n\n I didn't realize how barbaric it was to ...,M. L. Kijewski,5.0
7,B08267BBJT,Size: AAA,"July 1, 2020",Verified Purchase,\nEnergy Storage Is Less Than 1/4 Normal Batte...,"\n\n They do charge quickly, but at what pric...",Brent R.,1.0
8,B08267BBJT,Size: AAA,"June 28, 2020",Verified Purchase,\nPoor Quality - Unreliable\n,\n\n I had high hopes for these batteries bec...,Christian D.,1.0
9,B08267X3LH,Size: 9V,"June 12, 2020",Verified Purchase,\nGet these batteries!\n,\n\n I can’t believe that I can love a batter...,techie511,5.0


In [18]:
no_rev_content = no_rev.content
no_rev_content

b'<!doctype html><html lang="en-us" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n\n\n\n<script>var ue_t0=ue_t0||+new Date();</script>\n\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-na.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n\n<script>\n(function(a){var b=a.performance;a.ue_mark=b&&"function"===typeof b.mark?function(a){b.mark(a)}:function(){}})(window);\n\n    ue_mark(\'csm:ho:ob\');\n</script>\n<script>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\n\nif (window.ue_ihb === 1) {\n\n    var ue_csm = window,\n        ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.s

In [17]:
no_rev_soup = BeautifulSoup(no_rev_content)

In [22]:
no_rev_soup.find('div', attrs={'class': 'a-section a-spacing-top-large a-text-center no-reviews-section'}) == None

False

In [21]:
review_soup.find('div', attrs={'class': 'a-section a-spacing-top-large a-text-center no-reviews-section'}) == None

True

In [23]:
if no_rev_soup.find('div', attrs={'class': 'a-section a-spacing-top-large a-text-center no-reviews-section'}) == None:
    print("let's get that content")
else:
    print('NO REVIEWS')

NO REVIEWS


In [9]:
reviews = []

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

asin_list = ['B07S4B63Y5', 'B082W2MP8Y', 'B07XYZK7DX', 'B07MKM55NS', 'B07WMG47LW', 'B00HZV9WTM']

for asin in asin_list:
    passed_last_page = None
    counter = 1
    while passed_last_page == None:

        reviews_url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={counter}'
        rev = requests.get(reviews_url, headers=headers)
        reviews_page_content = rev.content

        review_soup = BeautifulSoup(reviews_page_content)
        passed_last_page = review_soup.find('div', attrs={'class': 'a-section a-spacing-top-large a-text-center no-reviews-section'})

        if passed_last_page == None:

            for d in review_soup.findAll('div', attrs={'data-hook':'review'}):
            #         print(d)
                    try:
                        date = d.find('span', attrs={'data-hook':'review-date'})
                        date = date.text.split(' ')[-3:]
                        date = ' '.join(date)
                    except:
                        date = 'null'
                    try:
                        title = d.find('a', attrs={'data-hook': 'review-title'})
                    except:
                        title = 'null'
                    try:
                        product = d.find('a', attrs={'data-hook': 'format-strip'})
                        product = product.text
                    except:
                        product = 'null'
                    try:
                        review_asin = product['href'].split('/')[3]
                    except:
                        review_asin = asin
                    try:
                        verified = d.find('span', attrs={'data-hook':'avp-badge'})
                        if verified == None:
                            verified = 'Not Verified'
                        else:
                            verified = verified.text
                    except:
                        verified = 'null'
                    try:
                        description = d.find('span', attrs={'data-hook': 'review-body'})
                    except:
                        description = 'null'
                    try:
                        reviewer_name = d.find('span', attrs={'class': 'a-profile-name'})
                    except:
                        reviewer_name = 'null'
                    try:
                        stars = d.find('span', attrs={'class': 'a-icon-alt'})
                    except:
                        stars = 'null'
                    reviews.append([review_asin, product, date, verified, title.text, description.text, reviewer_name.text, float(stars.text[0:3])])
        else:
            pass

        counter += 1

        time.sleep(15) # Sample randomly to add noise
    
reviews_df = pd.DataFrame(reviews, columns=['asin','product','date', 'verified', 'title', 'desc', 'reviewer_name', 'rating'])

KeyboardInterrupt: 

In [10]:
https://www.amazon.com/RAYOVAC-12-Pack-Alkaline-Batteries-813-12PPK/product-reviews/B00008MOQ7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews

1930

In [ ]:
https://www.amazon.com/RAYOVAC-12-Pack-Alkaline-Batteries-813-12PPK/product-reviews/B00008MOQ7/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2

In [12]:
reviews_df = pd.DataFrame(reviews, columns=['asin','product','date', 'verified', 'title', 'desc', 'reviewer_name', 'rating'])

In [13]:
reviews_df

,asin,product,date,verified,title,desc,reviewer_name,rating
0,B07XYZK7DX,Size: 1*9V,"June 20, 2020",Verified Purchase,\nGood job\n,\n\n I do question how long it'll last but I ...,Aaron,5.0
1,B07XYZK7DX,Size: 1*9V,"June 17, 2020",Verified Purchase,\nWorks Great\n,\n\n Bought this so I wouldn't have to keep g...,Kirk Keller,5.0
2,B07XYZK7DX,Size: 1*9V,"June 16, 2020",Verified Purchase,\nMore versatile because of not having to use ...,\n\n I do like being able to charge the batte...,Duane Lowe,5.0
3,B07XYZK7DX,Size: 1*9V,"June 1, 2020",Verified Purchase,\nSave your money\n,\n\n If you dont mind charging it every other...,Katrina L Reynosa,1.0
4,B07XYZK7DX,Size: 2*9V,"April 22, 2020",Verified Purchase,\nBrings a ton of convenience\n,\n\n Brings a ton of convenience when I am tr...,Alivya,5.0
...,...,...,...,...,...,...,...,...
1925,B00HZV9WTM,Style: BatteriesSize: AAA 8 Pack,"December 10, 2018",Verified Purchase,\nBatteries advertised as charged. Totally de...,\n\n Totally dead when I received the batteri...,William M.,2.0
1926,B00HZV9WTM,Style: BatteriesSize: AA 4 Pack,"December 10, 2018",Verified Purchase,\nGreat\n,\n\n Got the new silver ones. The older black...,David S.,5.0
1927,B00HZV9WTM,Style: BatteriesSize: AA 8 Pack,"December 10, 2018",Verified Purchase,\nGood batteries but quality control lacking\n,\n\n Out of 8 batteries in the pack i ordered...,Ethan Banks,5.0
1928,B00HZV9WTM,Style: BatteriesSize: AA 8 Pack,"December 9, 2018",Verified Purchase,\nGreat battery\n,\n\n Solid performance\n\n,stangcolts,5.0


In [14]:
reviews_df.to_csv('reviews_scrape_5.csv')

In [5]:
df = pd.read_csv('reviews_scrape.csv')

In [6]:
df.head()

,Unnamed: 0,asin,product,date,verified,title,desc,reviewer_name,rating
0,0,B08267BBJT,Size: AAA,"August 11, 2020",Verified Purchase,"\nDidn't work, then worked, now don't work aga...",\n\n All I got in terms of use out of these b...,Jasmine Carroll,1.0
1,1,B08267BBJT,Size: AAA,"July 30, 2020",Verified Purchase,\nThese absolutely suck\n,\n\n I bought these for a wall mounted magnif...,Ashlee M.,1.0
2,2,B08268F6XN,Size: AA,"July 19, 2020",Verified Purchase,\nlonger lasting battery for remote controller...,\n\n i like the constant voltage and hopefull...,ARCHANGEL TROY,5.0
3,3,B08267BBJT,Size: AAA,"July 18, 2020",Verified Purchase,\nMinimal plastic in packaging.\n,"\n\n Just received these today, but I’m revie...",ira,5.0
4,4,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,\nNot long enough battery life for a night hike\n,\n\n Shuts off suddenly in headlamp\n\n,T,3.0


In [7]:
df.shape

(1089, 9)

In [9]:
df[-15:-1]

,Unnamed: 0,asin,product,date,verified,title,desc,reviewer_name,rating
1074,1074,B072R2SWXX,Size: ONLY USB BATTERIES,"April 24, 2018",Verified Purchase,\nI would buy this again\n,"\n\n So, far so good. It has held the charge...",Urduja,5.0
1075,1075,B072R2SWXX,Size: ONLY USB BATTERIES,"April 14, 2018",Verified Purchase,\nEach battery charged out of the pack. With a...,\n\n Each battery charged out of the pack. W...,Amazon Customer,5.0
1076,1076,B072R2SWXX,Size: ONLY USB BATTERIES,"April 12, 2018",Verified Purchase,\nI love the USB AA Rechargeable batteries bec...,\n\n I love the USB AA Rechargeable batteries...,CPT Geek,5.0
1077,1077,B072R2SWXX,Size: ONLY USB BATTERIES,"April 11, 2018",Verified Purchase,\nLong life and solar rechargeable.\n,\n\n The long life and ease of use is very im...,Max,5.0
1078,1078,B072R2SWXX,Size: ONLY USB BATTERIES,"April 10, 2018",Verified Purchase,\nGreat design and great product\n,\n\n Great design and great product! Fit into...,B. Pham,5.0
1079,1079,B072R2SWXX,Size: ONLY USB BATTERIES,"April 10, 2018",Verified Purchase,\nFive Stars\n,"\n\n Great buy, Long lasting, convenient\n\n",William W.,5.0
1080,1080,B072R2SWXX,Size: ONLY USB BATTERIES,"April 10, 2018",Verified Purchase,\nFive Stars\n,\n\n Better than expected!\n\n,Amazon Customer,5.0
1081,1081,B072R2SWXX,Size: ONLY USB BATTERIES,"April 9, 2018",Verified Purchase,\nand last a good amount of time (about as lon...,\n\n Just what I needed for my Dell VR hand c...,X,5.0
1082,1082,B072R2SWXX,Size: ONLY USB BATTERIES,"April 4, 2018",Verified Purchase,\nGood idea\n,"\n\n I think this is a Good Idea, handy to ha...",Mig,5.0
1083,1083,B072R2SWXX,Size: ONLY USB BATTERIES,"April 2, 2018",Verified Purchase,\nThey work well\n,\n\n They work... and are rechargeable; it do...,Greene N,5.0


In [14]:
df[df['asin'] == 'B072R2SWXX']

,Unnamed: 0,asin,product,date,verified,title,desc,reviewer_name,rating
968,968,B072R2SWXX,Size: ONLY USB BATTERIES,"June 27, 2020",Verified Purchase,\nI'm impressed\n,"\n\n I haven't used them yet,the way they get...",Amazon Customer,5.0
969,969,B072R2SWXX,Size: ONLY USB BATTERIES,"June 8, 2020",Not Verified,\nThe add that this seller is advertising is m...,\n\n The add that this seller is advertising ...,Adisus,1.0
970,970,B072R2SWXX,Size: ONLY USB BATTERIES,"June 6, 2020",Verified Purchase,\nvery eazy\n,\n\n i loved these batteries so much i ended ...,Reid smolens,5.0
971,971,B072R2SWXX,Size: ONLY USB BATTERIES,"April 8, 2020",Verified Purchase,\nLong charges\n,\n\n Surprising how long the batteries stay c...,Al,5.0
972,972,B072R2SWXX,Size: ONLY USB BATTERIES,"April 7, 2020",Verified Purchase,\nBetter then the rest\n,\n\n This is the best design in my opinion. I...,Derek Smith,5.0
...,...,...,...,...,...,...,...,...,...
1084,1084,B072R2SWXX,Size: ONLY USB BATTERIES,"March 31, 2018",Not Verified,\nUnique AA Battery & USB charger\n,\n\n This is a unique quality battery that is...,Amazon Customer,5.0
1085,1085,B072R2SWXX,Size: ONLY USB BATTERIES,"March 30, 2018",Verified Purchase,\nIt’s great having these!\n,\n\n I have just gotten the 4 rechargeable AA...,Cee Kay,5.0
1086,1086,B072R2SWXX,Size: ONLY USB BATTERIES,"March 26, 2018",Verified Purchase,\nI bought the batteries as backup just in cas...,\n\n I bought the batteries as backup just in...,SC,5.0
1087,1087,B072R2SWXX,Size: ONLY USB BATTERIES,"March 23, 2018",Verified Purchase,\nFive Stars\n,"\n\n Awesome Products, I use them in my GPS a...",427Cobra,5.0


In [19]:
df2 = pd.read_csv('reviews_scrape_2.csv')

In [20]:
df2

,Unnamed: 0,asin,product,date,verified,title,desc,reviewer_name,rating
0,0,B0828KRQZ3,NaN,"August 20, 2020",Verified Purchase,\nCapacity a bit lower than listed.\n,\n\n I love EBL product however their batteri...,Andrey B.,3.0
1,1,B0828KRQZ3,NaN,"August 11, 2020",Verified Purchase,\nTrue 1.5v AA rechargeable batteries\n,\n\n true 1.5v AA rechargeables. I should hav...,Mr. Shin,5.0
2,2,B0828KRQZ3,NaN,"August 11, 2020",Verified Purchase,\nAbsolute garbage\n,\n\n So I bought these batteries hoping for s...,Amazon Customer,1.0
3,3,B0828KRQZ3,NaN,"August 11, 2020",Verified Purchase,"\nWorks in Blink XT2 Camera, Great Customer Su...",\n\n I purchased these batteries to use in my...,Dh1985,5.0
4,4,B0828KRQZ3,NaN,"August 10, 2020",Verified Purchase,\nGreat charging features\n,"\n\n These batteries are very high capacity, ...",Paul C,5.0
...,...,...,...,...,...,...,...,...,...
192,192,B07QV15B3W,Size: 2 Count (Pack of 1),"July 15, 2019",Verified Purchase,\nAwesome rechargeable batteries\n,\n\n I just bought a wireless mac keyboard an...,Adam,5.0
193,193,B07QV15B3W,Size: 2 Count (Pack of 1),"July 5, 2019",Verified Purchase,\nGreat idea\n,\n\n Rechargeable batteries without an awkwar...,Benjamin Trotter,5.0
194,194,B07QV15B3W,Size: 2 Count (Pack of 1),"July 2, 2019",Verified Purchase,\nThey came a long way\n,\n\n I remember the days where you just toss ...,Crafter,5.0
195,195,B07QV15B3W,Size: 2 Count (Pack of 1),"June 25, 2019",Not Verified,\nCharge lasts\n,\n\n These will help save money. They rechar...,Taxlady,5.0


In [21]:
df3 = df.append(df2)

In [22]:
df3

,Unnamed: 0,asin,product,date,verified,title,desc,reviewer_name,rating
0,0,B08267BBJT,Size: AAA,"August 11, 2020",Verified Purchase,"\nDidn't work, then worked, now don't work aga...",\n\n All I got in terms of use out of these b...,Jasmine Carroll,1.0
1,1,B08267BBJT,Size: AAA,"July 30, 2020",Verified Purchase,\nThese absolutely suck\n,\n\n I bought these for a wall mounted magnif...,Ashlee M.,1.0
2,2,B08268F6XN,Size: AA,"July 19, 2020",Verified Purchase,\nlonger lasting battery for remote controller...,\n\n i like the constant voltage and hopefull...,ARCHANGEL TROY,5.0
3,3,B08267BBJT,Size: AAA,"July 18, 2020",Verified Purchase,\nMinimal plastic in packaging.\n,"\n\n Just received these today, but I’m revie...",ira,5.0
4,4,B08267BBJT,Size: AAA,"July 17, 2020",Verified Purchase,\nNot long enough battery life for a night hike\n,\n\n Shuts off suddenly in headlamp\n\n,T,3.0
...,...,...,...,...,...,...,...,...,...
192,192,B07QV15B3W,Size: 2 Count (Pack of 1),"July 15, 2019",Verified Purchase,\nAwesome rechargeable batteries\n,\n\n I just bought a wireless mac keyboard an...,Adam,5.0
193,193,B07QV15B3W,Size: 2 Count (Pack of 1),"July 5, 2019",Verified Purchase,\nGreat idea\n,\n\n Rechargeable batteries without an awkwar...,Benjamin Trotter,5.0
194,194,B07QV15B3W,Size: 2 Count (Pack of 1),"July 2, 2019",Verified Purchase,\nThey came a long way\n,\n\n I remember the days where you just toss ...,Crafter,5.0
195,195,B07QV15B3W,Size: 2 Count (Pack of 1),"June 25, 2019",Not Verified,\nCharge lasts\n,\n\n These will help save money. They rechar...,Taxlady,5.0


In [41]:
check_list = ['B07HQ7QV7W', 'B083ZMYF55', 'B07FQD7PZ5', 'B07TWDR7VJ', 'B085DV7VZK', 'B086GTFGPP', 'B07Q6PZ2F4', 'B082W54KQK']
for asin in check_list:
    print(df3[df3['asin'] == asin].shape)

(0, 9)
(0, 9)
(0, 9)
(0, 9)
(0, 9)
(0, 9)
(0, 9)
(0, 9)


In [36]:
df4 = pd.read_csv('reviews_scrape_3.csv')

In [38]:
df5 = df3.append(df4)

In [40]:
df5.to_csv('total_reviews.csv')

In [8]:
check_list = ['B07S4B63Y5', 'B082W2MP8Y', 'B07XYZK7DX', 'B07MKM55NS', 'B07WMG47LW', 'B00HZV9WTM', '']
for asin in check_list:
    print(t[t['asin'] == asin].shape)

(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)


In [48]:
df6 = pd.read_csv('reviews_scrape_4.csv')

In [50]:
df6.shape

(900, 9)

In [51]:
df7 = df5.append(df6)

In [52]:
df7.shape

(2815, 9)

In [54]:
df7.to_csv('total_reviews.csv')

In [55]:
test_df = pd.read_csv('total_reviews.csv')